In [1]:
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import ElementTree
import requests
import re
import pandas as pd
import numpy as np

In [2]:
#http://131.243.229.11/setup/loggersetup.cgi?ADDRESS=42&REFRESH=Advanced, signal strength
#http://131.243.229.11/setup/devlist.cgi?SETUP=XML, device list
devices_xml = requests.get("http://131.243.229.11/setup/devlist.cgi?SETUP=XML", auth=('admin', '8o14i^Km')).text
tree = ET.ElementTree(ET.fromstring(devices_xml))

root = tree.getroot()
device_names = []

for address in root.iter('device'):
    device_names.append(address.attrib['address'])
    
device_names.remove('250')

In [3]:
#Use to test beautiful soup, need to filter maybe
test = requests.get("http://131.243.229.11/setup/loggersetup.cgi?ADDRESS=42&REFRESH=Advanced", 
                    auth=('admin', '8o14i^Km')).text
soup_test = BeautifulSoup(test)
print(soup_test.prettify())

<html>
 <head>
  <title>
  </title>
  <meta content="Stephen Herzog, for Obvius Holdings LLC" name="AUTHOR"/>
  <meta content="Copyright 2001-2016 by Obvius Holdings LLC, All Rights Reserved." name="COPYRIGHT"/>
  <meta content="SDProxy: portions Copyright 1999-2003 by Secure Design, All Rights Reserved." name="COPYRIGHT"/>
  <meta content="Network Setup Pages" name="KEYWORDS"/>
  <meta content="Setup pages for this server" name="DESCRIPTION"/>
  <meta content="Acquisuite cgi application" name="GENERATOR"/>
  <meta content="Sep 19 2016 15:51:51 Pacific" name="COMPILEDATE"/>
  <meta content="$Id: dev_r9120.cpp,v 1.87 2016/07/28 16:37:01 alevy Exp $" name="SOURCE"/>
  <meta content="width=device-width" name="viewport"/>
  <link href="/style.css" media="all" rel="stylesheet" type="text/css"/>
 </head>
 <body bgcolor="#FFFFFF" topmargin="5">
  <script language="JavaScript" src="relaybutton.js" type="text/javascript">
  </script>
  <form action="loggersetup.cgi?ADDRESS=42&amp;T=264B&amp;REF

In [6]:
data_frame = pd.DataFrame()

for device in device_names:
    html_string = requests.get("http://131.243.229.11/setup/loggersetup.cgi?ADDRESS=" + str(device) + "&REFRESH=Advanced",
                              auth=('admin', '8o14i^Km')).text
    text = BeautifulSoup(html_string).get_text()
    
    test_devices = device_names.copy()
    test_devices.remove(device)
    signals = []
    
    for line in text.splitlines():
        address = re.match(r'(Address:) ([0-9]+)', line)
        if address and address[2] in device_names:
            if re.search(r'[0-9]+%', line):
                signals.append(re.search(r'[0-9]+%', line)[0])
            else:
                signals.append('NaN')
            
    if len(signals)==0:
        data_frame[device] = np.full(len(test_devices) + 1, "NaN")
    else:
        column = pd.Series(signals, index=list(map(int, test_devices)))
        column = column.append(pd.Series(["100%"], index=[int(device)])).sort_index()
        data_frame[device] = column

In [7]:
#Nuances: When mapping an address to itself (42->42), I assume 100% signal strength. Also, devices where there is an error
#give back a NaN column. Also note that this is a column to row relationship, as in, I am only taking data from what is given
#from the respective column address data. For instance, if address 24 says its strength to 42 is 22%, that is true for column
#24 matched to row 42. That is not true for column 42 to row 24! 
data_frame

,24,25,28,29,30,31,32,33,34,36,37,40,41,42,43,44,48,49
24,100%,NaN,0%,1%,5%,NaN,2%,4%,0%,1%,NaN,2%,1%,18%,8%,3%,7%,1%
25,1%,NaN,0%,15%,0%,NaN,7%,7%,4%,4%,NaN,12%,4%,5%,3%,6%,4%,2%
28,1%,NaN,100%,1%,3%,NaN,8%,7%,4%,4%,NaN,1%,4%,5%,3%,1%,4%,2%
29,12%,NaN,0%,100%,16%,NaN,71%,9%,31%,2%,NaN,49%,7%,1%,19%,37%,1%,1%
30,10%,NaN,0%,19%,100%,NaN,51%,12%,33%,27%,NaN,15%,23%,1%,15%,33%,36%,1%
31,10%,NaN,0%,29%,NaN,NaN,51%,35%,33%,27%,NaN,15%,23%,28%,29%,33%,36%,24%
32,19%,NaN,0%,58%,47%,NaN,100%,72%,47%,55%,NaN,3%,54%,53%,55%,52%,70%,45%
33,4%,NaN,0%,6%,5%,NaN,73%,100%,8%,2%,NaN,21%,12%,4%,16%,17%,3%,1%
34,9%,NaN,0%,28%,32%,NaN,50%,12%,100%,1%,NaN,1%,32%,1%,18%,26%,35%,22%
36,10%,NaN,0%,1%,1%,NaN,56%,3%,1%,100%,NaN,15%,1%,9%,30%,1%,2%,2%


In [ ]:
data_frame = data_frame.reset_index()
data_frame.to_csv("signal_strength.csv", index=False)